---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [26]:
### LOADING DATA AND IMPORTING LIBRARIES
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from adspy_shared_utilities import plot_class_regions_for_classifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import hashlib
from numpy import random


train = pd.read_csv('train.csv', encoding = 'ISO-8859-1') # reading data. The files are not in utf8 format so we need to use encoding = 'ISO-8859-1' in order to read them properly 
test = pd.read_csv('test.csv', encoding = 'ISO-8859-1' )
address = pd.read_csv('addresses.csv')
latlons = pd.read_csv('latlons.csv')

temp = train.dropna(subset=(['compliance'])) # here I drop all rows which the 'compliance' feature is NaN.     

#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
a=pd.merge(address,latlons,on='address')  # merging address and latlons so we have all latlons corresponds to each address and ticket_id
b=pd.merge(a,temp,on='ticket_id') # in temp (which is the same as train without rows of NaN values for 'compliance) we add corresponding latlons to each ticket_id

############################################
b1=pd.merge(a,test,on='ticket_id')
############################################

address_related_columns=['address','violation_street_number','violation_street_name','mailing_address_str_number',
                         'mailing_address_str_name', 'zip_code','non_us_str_code']
b.drop(address_related_columns,axis=1,inplace=True) #remove all address related columns in b and only keep latlons data for address

###########################################
b1.drop(address_related_columns,axis=1,inplace=True)
###########################################


b = b.dropna(subset=(['lat','lon']))

###########################################
#b1 = b1.dropna(subset=(['lat','lon']))
b1.lat.fillna(b1.lat.mean(),inplace=True)
b1.lon.fillna(b1.lon.mean(),inplace=True)

###########################################

test_columns = [item for item in test.columns]
b_columns = [item for item in b.columns]

###########################################
b1_columns = [item for item in b1.columns]
###########################################

intersect_columns = [val for val in test_columns if val in b_columns] # list of columns that are in common in test and b 

###########################################
intersect_columns = [val for val in test_columns if val in b_columns]
###########################################


columns_to_keep = [val for val in test_columns if val in b_columns] # list of columns that are in common in test and b
columns_to_keep.append('lat')
columns_to_keep.append('lon')
X = b[columns_to_keep] # here we remove all columns which are not in 'test' to avoid data leackage except lat&lon which we need them

############################################
X_t = b1[columns_to_keep]
############################################


X.drop(['violation_zip_code','grafitti_status'], axis=1, inplace=True) # we remove these two columns by hand because all values of them are NaN

#############################################
X_t.drop(['violation_zip_code','grafitti_status'], axis=1, inplace=True)
#############################################

y = b['compliance'] # corresponding values of 'compliance' for each row in X

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [27]:
''' Removing the features which are more frequent than 20% of the whole data. For example 'violator_name' is almost a 
    Value which seems not to be useful in training process'''
entire_data_length=X.shape[0]/5
droped_columns=[]
for i in range(len(X.columns[:-2])): #here we exlude last two columns to be remove since they are lat and lon which we need later
    if len(X.drop_duplicates(X.columns[i]))>entire_data_length:
        droped_columns.append([X.columns[i]][0])

droped_columns_ticket_id = droped_columns.copy()
droped_columns.remove('ticket_id')
X.drop(droped_columns_ticket_id,axis=1,inplace=True)
X_t.drop(droped_columns,axis=1,inplace=True)

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [28]:
# here we apply the previous function to map all test phrases (such as names, explanations, ...) to numbers
for i in X.columns:
    if type(X[i][0])==str:
        X[i] = X[i].apply(lambda x: round((int(hashlib.sha1(str(x).encode('utf-8')).hexdigest(),16))/(10e43)))
        X_t[i] = X_t[i].apply(lambda x: round((int(hashlib.sha1(str(x).encode('utf-8')).hexdigest(),16))/(10e43)))
        

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [29]:
# here we map all lat and lon into 100 different segmentation 
segment=100
X['lat']=round(X['lat']*segment)
X['lon']=round(X['lon']*segment)
X_t['lat']=round(X_t['lat']*segment)
X_t['lon']=round(X_t['lon']*segment)

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

### Hereafter we try different algorythm to perform on our data to see which one gives better results

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
#scaler = MinMaxScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

#clf = LogisticRegression().fit(X_train, y_train)
#clf = RandomForestClassifier().fit(X_train, y_train)                                    # 0.99 0.94 - FP and FN [2048,447]
#clf = GaussianNB().fit(X_train, y_train)                                                # 0.89 0.89 - FP and FN [1966,2474]
clf = GradientBoostingClassifier().fit(X_train, y_train)                                #0.94 0.94 FP and FN [2212 66]
#clf = GradientBoostingClassifier(learning_rate = 0.01, max_depth = 2, random_state = 0).fit(X_train, y_train) 
                                                                                         #0.93 0.93 FP and FN [2692 14]
#clf = GradientBoostingClassifier(learning_rate = 0.1, max_depth = 3, random_state = 0).fit(X_train, y_train)    
                                                                                         #0.94 0.94 FP and FN [2212 66]
#clf = GradientBoostingClassifier(learning_rate = 0.2, max_depth = 6, random_state = 0).fit(X_train, y_train) 
                                                                                        #0.98 0.94 FP and FN [2150 169]
#clf = DecisionTreeClassifier(max_depth=8).fit(X_train, y_train)
#y_proba = clf.fit(X_train, y_train).predict_proba(X_test)

#y_proba_list = list(zip(y_test[0:20], y_proba[0:20,1]))

y_score = clf.fit(X_train, y_train).decision_function(X_test)
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)
print(roc_auc)

clf_predicted = clf.predict(X_test)
confusion = confusion_matrix(y_test, clf_predicted)

#print('Accuracy of RF classifier on training set: {:.2f}'
#     .format(clf.score(X_train, y_train)))
#print('Accuracy of RF classifier on test set: {:.2f}'
#     .format(clf.score(X_test, y_test)))
#print('Logistic regression classifier (default settings)\n', confusion)

0.780235659544


In [60]:
# the order of X_t was disturbed due to data preparation in previous steps. Here we re-order the X_t based on the order of
# ticket_id in original test.csv which is the same order as the autograder wants
X_t = X_t.set_index('ticket_id')
X_t = X_t.reindex(index=test['ticket_id'])
X_t = X_t.reset_index()


In [61]:
X_t_without_ticket_id=X_t.copy()
X_t_without_ticket_id.drop(['ticket_id'],axis=1,inplace=True)
#X_t_without_ticket_id.head()
#X_t.head()

In [62]:
def blight_model():
    
    # Your code here
    
    y_proba = clf.predict_proba(X_t_without_ticket_id)
    ans = pd.Series(y_proba[:,1], index=test.ticket_id)
    
    return ans


In [63]:
blight_model()

ticket_id
284932    0.120077
285362    0.015775
285361    0.051208
285338    0.060499
285346    0.087147
285345    0.073806
285347    0.091039
285342    0.900067
285530    0.022974
284989    0.023882
285344    0.069127
285343    0.024613
285340    0.019437
285341    0.057753
285349    0.079705
285348    0.067419
284991    0.024104
285532    0.023363
285406    0.018113
285001    0.037639
285006    0.024893
285405    0.020071
285337    0.020073
285496    0.051201
285497    0.052111
285378    0.016958
285589    0.018963
285585    0.042661
285501    0.054260
285581    0.015607
            ...   
376367    0.019555
376366    0.032303
376362    0.178950
376363    0.193546
376365    0.019555
376364    0.032303
376228    0.045374
376265    0.030531
376286    0.242345
376320    0.039537
376314    0.027603
376327    0.371025
376385    0.301046
376435    0.903175
376370    0.926049
376434    0.061721
376459    0.050667
376478    0.014365
376473    0.033468
376484    0.026847
376482    0.022945
37